# ***Step 1: Import libraries & Set up***

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 12 # N_mffc
hidden_size = 64
num_layers = 3
num_classes = 35
batch_size = 1
num_epochs = 25
learning_rate = 0.001

# ***Step 2: Use trained model (FP32)***
Reference: https://github.com/felixchenfy/Speech-Commands-Classification-by-LSTM-PyTorch

In [ ]:
# Recurrent neural network (many-to-one)
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        return x
model = LSTM(12, 35,64,3)
x = torch.randn(batch_size, sequence_length, input_size)
out = model(x)
print(out)



tensor([[[ 1.7270e-01,  8.3038e-01,  9.7014e-01,  4.0687e-01, -1.4660e+00,
           3.0980e-01, -3.5015e-01, -1.0706e+00, -7.7508e-01, -3.3848e-01,
          -1.2167e+00,  3.4661e-02],
         [-1.3843e+00, -2.5632e-01, -2.0668e+00,  7.6083e-04,  2.9271e-01,
           6.3341e-01,  1.0483e-02, -5.5824e-01,  2.4236e+00, -1.0725e+00,
          -2.0279e-01, -1.6951e+00],
         [-2.0167e+00, -5.4226e-01,  8.2412e-01,  1.7078e+00,  1.1082e-01,
          -6.9136e-01,  9.2024e-01,  3.0344e-01,  1.7154e+00, -7.7499e-01,
          -8.5368e-01,  1.1135e+00],
         [ 5.0654e-01, -3.1274e-01,  1.4626e-01, -2.0220e-01,  5.0117e-01,
           1.9695e-02,  1.2676e+00, -1.6671e-01,  2.1423e+00,  1.3618e-01,
          -1.4192e+00, -1.3121e-01],
         [-2.1804e+00, -1.2250e-01, -7.1894e-01,  1.5719e+00, -1.1279e+00,
           1.7011e+00,  1.1586e+00, -9.7343e-01,  5.7428e-01,  2.5672e-01,
           1.3733e+00, -3.9467e-01],
         [ 4.3122e-01, -4.7075e-01,  1.3595e+00,  1.0747e-01,  1.

***--- Above LSTM model tensors representation for Affine per tensor quantization--***

In [ ]:
import os

In [ ]:
T = torch.tensor(out)
print(T)

tensor([[[ 1.7270e-01,  8.3038e-01,  9.7014e-01,  4.0687e-01, -1.4660e+00,
           3.0980e-01, -3.5015e-01, -1.0706e+00, -7.7508e-01, -3.3848e-01,
          -1.2167e+00,  3.4661e-02],
         [-1.3843e+00, -2.5632e-01, -2.0668e+00,  7.6083e-04,  2.9271e-01,
           6.3341e-01,  1.0483e-02, -5.5824e-01,  2.4236e+00, -1.0725e+00,
          -2.0279e-01, -1.6951e+00],
         [-2.0167e+00, -5.4226e-01,  8.2412e-01,  1.7078e+00,  1.1082e-01,
          -6.9136e-01,  9.2024e-01,  3.0344e-01,  1.7154e+00, -7.7499e-01,
          -8.5368e-01,  1.1135e+00],
         [ 5.0654e-01, -3.1274e-01,  1.4626e-01, -2.0220e-01,  5.0117e-01,
           1.9695e-02,  1.2676e+00, -1.6671e-01,  2.1423e+00,  1.3618e-01,
          -1.4192e+00, -1.3121e-01],
         [-2.1804e+00, -1.2250e-01, -7.1894e-01,  1.5719e+00, -1.1279e+00,
           1.7011e+00,  1.1586e+00, -9.7343e-01,  5.7428e-01,  2.5672e-01,
           1.3733e+00, -3.9467e-01],
         [ 4.3122e-01, -4.7075e-01,  1.3595e+00,  1.0747e-01,  1.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


***--Maximum Value and minimum value of x***

In [ ]:
# Min-Max value of float_32 tensor (x) find out for scale (s) and zero point (z)

b = torch.max(T)
print(b)

tensor(2.7507)


In [ ]:
a = torch.min(T)
print(a)

tensor(-2.9663)


In [ ]:
# scale value

s = (b-a)/15

print(s)

tensor(0.3811)


In [ ]:
# zero point

z = torch.round(-a*15/(b-a))

print(z)

tensor(8.)


# ***Method-1:- Asymmetric (Affine) quantization scheme***

# ***Step-3: Apply Quantization by round and clipping function (Affine mapping)***

In [ ]:
f = torch.round(T/s + z)
print(f)
Tq = torch.clip(f, min=0, max=15) # Here min & max value we can change as per Tbit.
# But, I have checked for 4 bit
print (Tq)
torch.save(Tq,'qtz_tensor.pt')


tensor([[[ 8., 10., 11.,  9.,  4.,  9.,  7.,  5.,  6.,  7.,  5.,  8.],
         [ 4.,  7.,  3.,  8.,  9., 10.,  8.,  7., 14.,  5.,  7.,  4.],
         [ 3.,  7., 10., 12.,  8.,  6., 10.,  9., 13.,  6.,  6., 11.],
         [ 9.,  7.,  8.,  7.,  9.,  8., 11.,  8., 14.,  8.,  4.,  8.],
         [ 2.,  8.,  6., 12.,  5., 12., 11.,  5., 10.,  9., 12.,  7.],
         [ 9.,  7., 12.,  8., 11.,  8., 10.,  8.,  7.,  8.,  9.,  4.],
         [ 7.,  9.,  6., 10., 10.,  9.,  5.,  8.,  5., 11., 12., 13.],
         [ 5., 12., 11., 13., 11.,  4., 11.,  8.,  4.,  8.,  9.,  8.],
         [ 6., 10.,  5.,  3.,  6.,  7.,  3., 15.,  5.,  4.,  7.,  7.],
         [ 6.,  8.,  4.,  9., 13.,  9., 11.,  9.,  8., 11.,  3.,  8.],
         [ 9., 12., 11.,  7., 10.,  9.,  5.,  7.,  4.,  8.,  7.,  6.],
         [ 7., 11., 10.,  5.,  7., 10.,  6.,  9.,  9.,  7.,  9.,  8.],
         [10., 11.,  8., 10.,  6.,  7.,  7.,  5., 10.,  5.,  5.,  6.],
         [ 8.,  4.,  6.,  6.,  7., 10.,  3.,  3.,  8.,  6., 10.,  4.],
      

# ***Step-4: Apply Dequantization***

In [ ]:
Tdq = s*(Tq - z)
print(Tdq)

tensor([[[ 0.0000,  0.7623,  1.1434,  0.3811, -1.5245,  0.3811, -0.3811,
          -1.1434, -0.7623, -0.3811, -1.1434,  0.0000],
         [-1.5245, -0.3811, -1.9057,  0.0000,  0.3811,  0.7623,  0.0000,
          -0.3811,  2.2868, -1.1434, -0.3811, -1.5245],
         [-1.9057, -0.3811,  0.7623,  1.5245,  0.0000, -0.7623,  0.7623,
           0.3811,  1.9057, -0.7623, -0.7623,  1.1434],
         [ 0.3811, -0.3811,  0.0000, -0.3811,  0.3811,  0.0000,  1.1434,
           0.0000,  2.2868,  0.0000, -1.5245,  0.0000],
         [-2.2868,  0.0000, -0.7623,  1.5245, -1.1434,  1.5245,  1.1434,
          -1.1434,  0.7623,  0.3811,  1.5245, -0.3811],
         [ 0.3811, -0.3811,  1.5245,  0.0000,  1.1434,  0.0000,  0.7623,
           0.0000, -0.3811,  0.0000,  0.3811, -1.5245],
         [-0.3811,  0.3811, -0.7623,  0.7623,  0.7623,  0.3811, -1.1434,
           0.0000, -1.1434,  1.1434,  1.5245,  1.9057],
         [-1.1434,  1.5245,  1.1434,  1.9057,  1.1434, -1.5245,  1.1434,
           0.0000, -1.52

# ***Step-5: MAE/MSE loss between T and Tdq***

***I. MAE loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(T, Tdq)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[[ 1.7270e-01,  8.3038e-01,  9.7014e-01,  4.0687e-01, -1.4660e+00,
           3.0980e-01, -3.5015e-01, -1.0706e+00, -7.7508e-01, -3.3848e-01,
          -1.2167e+00,  3.4661e-02],
         [-1.3843e+00, -2.5632e-01, -2.0668e+00,  7.6083e-04,  2.9271e-01,
           6.3341e-01,  1.0483e-02, -5.5824e-01,  2.4236e+00, -1.0725e+00,
          -2.0279e-01, -1.6951e+00],
         [-2.0167e+00, -5.4226e-01,  8.2412e-01,  1.7078e+00,  1.1082e-01,
          -6.9136e-01,  9.2024e-01,  3.0344e-01,  1.7154e+00, -7.7499e-01,
          -8.5368e-01,  1.1135e+00],
         [ 5.0654e-01, -3.1274e-01,  1.4626e-01, -2.0220e-01,  5.0117e-01,
           1.9695e-02,  1.2676e+00, -1.6671e-01,  2.1423e+00,  1.3618e-01,
          -1.4192e+00, -1.3121e-01],
         [-2.1804e+00, -1.2250e-01, -7.1894e-01,  1.5719e+00, -1.1279e+00,
           1.7011e+00,  1.1586e+00, -9.7343e-01,  5.7428e-01,  2.5672e-01,
           1.3733e+00, -3.9467e-01],
         [ 4.3122e-01, -4.7075e-01,  1.3595e+00,  

***II. MSE Loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(T, Tdq)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[[ 1.7270e-01,  8.3038e-01,  9.7014e-01,  4.0687e-01, -1.4660e+00,
           3.0980e-01, -3.5015e-01, -1.0706e+00, -7.7508e-01, -3.3848e-01,
          -1.2167e+00,  3.4661e-02],
         [-1.3843e+00, -2.5632e-01, -2.0668e+00,  7.6083e-04,  2.9271e-01,
           6.3341e-01,  1.0483e-02, -5.5824e-01,  2.4236e+00, -1.0725e+00,
          -2.0279e-01, -1.6951e+00],
         [-2.0167e+00, -5.4226e-01,  8.2412e-01,  1.7078e+00,  1.1082e-01,
          -6.9136e-01,  9.2024e-01,  3.0344e-01,  1.7154e+00, -7.7499e-01,
          -8.5368e-01,  1.1135e+00],
         [ 5.0654e-01, -3.1274e-01,  1.4626e-01, -2.0220e-01,  5.0117e-01,
           1.9695e-02,  1.2676e+00, -1.6671e-01,  2.1423e+00,  1.3618e-01,
          -1.4192e+00, -1.3121e-01],
         [-2.1804e+00, -1.2250e-01, -7.1894e-01,  1.5719e+00, -1.1279e+00,
           1.7011e+00,  1.1586e+00, -9.7343e-01,  5.7428e-01,  2.5672e-01,
           1.3733e+00, -3.9467e-01],
         [ 4.3122e-01, -4.7075e-01,  1.3595e+00,  

# ***Method-2:- Symmetric quantization scheme***

In [ ]:
f = torch.round(T/s + z)
print(f)
Tq1 = torch.clip(f, min=-8, max=8) # Here min & max value we can change as per Tbit.
# But, I have checked for 4 bit
print (Tq1)
torch.save(Tq1,'qtz_tensor.pt')


tensor([[[ 8., 10., 11.,  9.,  4.,  9.,  7.,  5.,  6.,  7.,  5.,  8.],
         [ 4.,  7.,  3.,  8.,  9., 10.,  8.,  7., 14.,  5.,  7.,  4.],
         [ 3.,  7., 10., 12.,  8.,  6., 10.,  9., 13.,  6.,  6., 11.],
         [ 9.,  7.,  8.,  7.,  9.,  8., 11.,  8., 14.,  8.,  4.,  8.],
         [ 2.,  8.,  6., 12.,  5., 12., 11.,  5., 10.,  9., 12.,  7.],
         [ 9.,  7., 12.,  8., 11.,  8., 10.,  8.,  7.,  8.,  9.,  4.],
         [ 7.,  9.,  6., 10., 10.,  9.,  5.,  8.,  5., 11., 12., 13.],
         [ 5., 12., 11., 13., 11.,  4., 11.,  8.,  4.,  8.,  9.,  8.],
         [ 6., 10.,  5.,  3.,  6.,  7.,  3., 15.,  5.,  4.,  7.,  7.],
         [ 6.,  8.,  4.,  9., 13.,  9., 11.,  9.,  8., 11.,  3.,  8.],
         [ 9., 12., 11.,  7., 10.,  9.,  5.,  7.,  4.,  8.,  7.,  6.],
         [ 7., 11., 10.,  5.,  7., 10.,  6.,  9.,  9.,  7.,  9.,  8.],
         [10., 11.,  8., 10.,  6.,  7.,  7.,  5., 10.,  5.,  5.,  6.],
         [ 8.,  4.,  6.,  6.,  7., 10.,  3.,  3.,  8.,  6., 10.,  4.],
      

In [ ]:
Tdq1 = s*(Tq1 - z)
print(Tdq1)

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000, -1.5245,  0.0000, -0.3811,
          -1.1434, -0.7623, -0.3811, -1.1434,  0.0000],
         [-1.5245, -0.3811, -1.9057,  0.0000,  0.0000,  0.0000,  0.0000,
          -0.3811,  0.0000, -1.1434, -0.3811, -1.5245],
         [-1.9057, -0.3811,  0.0000,  0.0000,  0.0000, -0.7623,  0.0000,
           0.0000,  0.0000, -0.7623, -0.7623,  0.0000],
         [ 0.0000, -0.3811,  0.0000, -0.3811,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000, -1.5245,  0.0000],
         [-2.2868,  0.0000, -0.7623,  0.0000, -1.1434,  0.0000,  0.0000,
          -1.1434,  0.0000,  0.0000,  0.0000, -0.3811],
         [ 0.0000, -0.3811,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000, -0.3811,  0.0000,  0.0000, -1.5245],
         [-0.3811,  0.0000, -0.7623,  0.0000,  0.0000,  0.0000, -1.1434,
           0.0000, -1.1434,  0.0000,  0.0000,  0.0000],
         [-1.1434,  0.0000,  0.0000,  0.0000,  0.0000, -1.5245,  0.0000,
           0.0000, -1.52

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq1)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(T, Tdq1)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[[ 1.7270e-01,  8.3038e-01,  9.7014e-01,  4.0687e-01, -1.4660e+00,
           3.0980e-01, -3.5015e-01, -1.0706e+00, -7.7508e-01, -3.3848e-01,
          -1.2167e+00,  3.4661e-02],
         [-1.3843e+00, -2.5632e-01, -2.0668e+00,  7.6083e-04,  2.9271e-01,
           6.3341e-01,  1.0483e-02, -5.5824e-01,  2.4236e+00, -1.0725e+00,
          -2.0279e-01, -1.6951e+00],
         [-2.0167e+00, -5.4226e-01,  8.2412e-01,  1.7078e+00,  1.1082e-01,
          -6.9136e-01,  9.2024e-01,  3.0344e-01,  1.7154e+00, -7.7499e-01,
          -8.5368e-01,  1.1135e+00],
         [ 5.0654e-01, -3.1274e-01,  1.4626e-01, -2.0220e-01,  5.0117e-01,
           1.9695e-02,  1.2676e+00, -1.6671e-01,  2.1423e+00,  1.3618e-01,
          -1.4192e+00, -1.3121e-01],
         [-2.1804e+00, -1.2250e-01, -7.1894e-01,  1.5719e+00, -1.1279e+00,
           1.7011e+00,  1.1586e+00, -9.7343e-01,  5.7428e-01,  2.5672e-01,
           1.3733e+00, -3.9467e-01],
         [ 4.3122e-01, -4.7075e-01,  1.3595e+00,  

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq1)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(T, Tdq1)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[[ 1.7270e-01,  8.3038e-01,  9.7014e-01,  4.0687e-01, -1.4660e+00,
           3.0980e-01, -3.5015e-01, -1.0706e+00, -7.7508e-01, -3.3848e-01,
          -1.2167e+00,  3.4661e-02],
         [-1.3843e+00, -2.5632e-01, -2.0668e+00,  7.6083e-04,  2.9271e-01,
           6.3341e-01,  1.0483e-02, -5.5824e-01,  2.4236e+00, -1.0725e+00,
          -2.0279e-01, -1.6951e+00],
         [-2.0167e+00, -5.4226e-01,  8.2412e-01,  1.7078e+00,  1.1082e-01,
          -6.9136e-01,  9.2024e-01,  3.0344e-01,  1.7154e+00, -7.7499e-01,
          -8.5368e-01,  1.1135e+00],
         [ 5.0654e-01, -3.1274e-01,  1.4626e-01, -2.0220e-01,  5.0117e-01,
           1.9695e-02,  1.2676e+00, -1.6671e-01,  2.1423e+00,  1.3618e-01,
          -1.4192e+00, -1.3121e-01],
         [-2.1804e+00, -1.2250e-01, -7.1894e-01,  1.5719e+00, -1.1279e+00,
           1.7011e+00,  1.1586e+00, -9.7343e-01,  5.7428e-01,  2.5672e-01,
           1.3733e+00, -3.9467e-01],
         [ 4.3122e-01, -4.7075e-01,  1.3595e+00,  